In [1]:
import pandas as pd

In [2]:
df_lables=pd.read_csv("dataset/operations_labels_training.csv")
df_train=pd.read_csv("dataset/telemetry_for_operations_training.csv")

In [3]:

# Check if the format is 'YYYY-MM-DD HH:MM:SS'
def check_format(date_str):
    # Try to convert to datetime with a specific format
    try:
        pd.to_datetime(date_str, format='%Y-%m-%d %H:%M:%S')
        return 'datetime'
    except ValueError:
        try:
            pd.to_datetime(date_str, format='%Y-%m-%d')
            return 'date'
        except ValueError:
            return 'invalid'

# Apply the function to the 'create_dt' column
df_train['format'] = df_train['create_dt'].apply(check_format)

In [4]:
# Display the rows with 'date' or 'invalid' format
df_train=df_train.drop(df_train[df_train['format'] != 'datetime'].index)


In [5]:
df_train['mdm_object_name']

0         1498
1         1498
2         1498
3         1498
4         1498
          ... 
516068     119
516069     119
516070     119
516071     119
516072     119
Name: mdm_object_name, Length: 516062, dtype: int64

In [6]:
def assign_labels(group):
    object_labels = df_lables[df_lables['mdm_object_name'] == group.name]
    
    for _, row in object_labels.iterrows():
        mask = (group['create_dt'] > row['start_time']) & (group['create_dt'] < row['end_time'])
        group.loc[mask, 'operation_kind_id'] = row['operation_kind_id']
        group.loc[mask, 'name'] = row['name']
    
    return group

# Apply the function to each group of mdm_object_name
train = df_train.groupby('mdm_object_name').apply(assign_labels)

# Verify the results
print(train['operation_kind_id'].value_counts(dropna=False))

# Save the updated DataFrame
train.to_csv('train_with_labels.csv', index=False)

/tmp/ipykernel_21140/3550935515.py:12: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  train = df_train.groupby('mdm_object_name').apply(assign_labels)


operation_kind_id
2.0    235243
5.0    162838
0.0     69217
1.0     30780
NaN     12968
3.0      5016
Name: count, dtype: int64
